<a href="https://colab.research.google.com/github/swoo-nam/project_final_team1/blob/main/%EC%9C%A0%EC%98%88%EC%8A%AC_yogiyo%EB%A9%80%ED%8B%B0%EB%9D%BC%EB%B2%A8%2B_%EA%B8%8D%EB%B6%80%EC%A0%95_%EB%B6%84%EC%84%9D%EB%AA%A8%EB%8D%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A. 멀티라벨 모델

In [61]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [62]:
import pandas as pd
import numpy as np
import re
import torch
from tqdm.auto import tqdm
import random
import os

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

# DATA_PATH = "/content/drive/MyDrive/생성 AI 모델링/data/"
DATA_PATH = "/content/drive/MyDrive/멀티캠퍼스 자료/Machine Learning/data/"
SEED = 42

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device


'cuda'

In [63]:
yogiyo =pd.read_csv(f"{DATA_PATH}yogiyo_reviews_jsi_all.csv")


In [64]:
# google_food = pd.read_csv(f"{DATA_PATH}playstore_food_reply_0904.csv")

In [65]:
# yogiyo_added =pd.read_csv(f"{DATA_PATH}yogiyo_reviews_0905.csv")


In [66]:
train= yogiyo[['고객리뷰','사장댓글']]

rename_cols = ['review', 'reply']
train.columns = rename_cols
train.head()

,review,reply
0,사진보다 실물이 백배 맛있어요~~~~~~ 고기 양도 많아요~~~~~~ 배달시간도 딱...,"이렇게 저희 매장 찾아주셔서 감사드립니다!!\n\n맛 뿐만 아니라 양도, 서비스도 ..."
1,리뷰가 늦었네요.. 맛있게 잘 먹었습니다. 또 시킬게요!!,좋은 리뷰 덕분에 기분이 행복해지는 것 같아요😇\n\n고객님께서 맛있다고 말씀 주시...
2,잘 먹었습니다. 다만 주문에 있던 펩시콜라는 받지 못했습니다.,소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. \n\n저희를 찾아 주신...
3,굿 말이 필요 없는 메뉴 였습니다,수 많은 매장들을 헤치고 와주셔서 감사합니다!!\n\n저희 매장의 메뉴를 제대로 즐...
4,맛있어요 양도많구좋음,수 많은 매장들을 헤치고 와주셔서 감사합니다😂\n\n메뉴 그 자체로 행복을 드리고 ...


In [67]:
review_list = []
for review in train['review']:
    if isinstance(review, str):

        pat = re.compile("[^a-zA-Z가-힣0-9.,~?!]")
        result = pat.sub(" ", review)

        result = re.sub(r'\.{2,}', '.', result)
        result = re.sub(r'\,{2,}', ',', result)
        result = re.sub(r'\~{2,}', '~', result)
        result = re.sub(r'\!{2,}', '!', result)
        result = re.sub(r'\?{2,}', '?', result)
        result = re.sub(" +", " ", result)
        review_list.append(result)
    else:
        review_list.append("")

train['review'] = review_list
train.head()

<ipython-input-67-6374a3c305ac>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['review'] = review_list


,review,reply
0,사진보다 실물이 백배 맛있어요~ 고기 양도 많아요~ 배달시간도 딱 맞춰서 보내주셨어요~,"이렇게 저희 매장 찾아주셔서 감사드립니다!!\n\n맛 뿐만 아니라 양도, 서비스도 ..."
1,리뷰가 늦었네요. 맛있게 잘 먹었습니다. 또 시킬게요!,좋은 리뷰 덕분에 기분이 행복해지는 것 같아요😇\n\n고객님께서 맛있다고 말씀 주시...
2,잘 먹었습니다. 다만 주문에 있던 펩시콜라는 받지 못했습니다.,소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. \n\n저희를 찾아 주신...
3,굿 말이 필요 없는 메뉴 였습니다,수 많은 매장들을 헤치고 와주셔서 감사합니다!!\n\n저희 매장의 메뉴를 제대로 즐...
4,맛있어요 양도많구좋음,수 많은 매장들을 헤치고 와주셔서 감사합니다😂\n\n메뉴 그 자체로 행복을 드리고 ...


In [68]:
reply_list = []
for reply in train['reply']:
    if isinstance(reply, str):

        pat = re.compile("[^a-zA-Z가-힣0-9.,~?!]")
        result = pat.sub(" ", reply)

        result = re.sub(r'\.{2,}', '.', result)
        result = re.sub(r'\,{2,}', ',', result)
        result = re.sub(r'\~{2,}', '~', result)
        result = re.sub(r'\!{2,}', '!', result)
        result = re.sub(r'\?{2,}', '?', result)
        result = re.sub(" +", " ", result)
        reply_list.append(result)
    else:
        reply_list.append("")

train['reply'] = reply_list
train.head()

<ipython-input-68-2764c488e75c>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['reply'] = reply_list


,review,reply
0,사진보다 실물이 백배 맛있어요~ 고기 양도 많아요~ 배달시간도 딱 맞춰서 보내주셨어요~,"이렇게 저희 매장 찾아주셔서 감사드립니다! 맛 뿐만 아니라 양도, 서비스도 모두 최..."
1,리뷰가 늦었네요. 맛있게 잘 먹었습니다. 또 시킬게요!,좋은 리뷰 덕분에 기분이 행복해지는 것 같아요 고객님께서 맛있다고 말씀 주시니 피로...
2,잘 먹었습니다. 다만 주문에 있던 펩시콜라는 받지 못했습니다.,소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. 저희를 찾아 주신 만큼 ...
3,굿 말이 필요 없는 메뉴 였습니다,수 많은 매장들을 헤치고 와주셔서 감사합니다! 저희 매장의 메뉴를 제대로 즐기신 거...
4,맛있어요 양도많구좋음,수 많은 매장들을 헤치고 와주셔서 감사합니다 메뉴 그 자체로 행복을 드리고 싶었는데...


## 키워드 추출 (Keybert + Kiwi)
- 리뷰별로 kiwi로 추출한 형태소 (명사류, 형용사, 어근)에서 keybert 키워드 추출
- keybert 키워드 pool 중에서 top 100 키워드 선정, 직접 5개 카테고리('맛, 양, 가격, 서비스, 배달')에 관한 키워드를 추출함
- 각 리뷰별로 kiwi 분석 결과에 키워드가 있는지에 따라 5개 카테고리 멀티 분류

In [69]:
!pip install kiwipiepy


In [70]:
!pip install keybert

In [71]:
from kiwipiepy import Kiwi

kiwi = Kiwi()


- 키워드 품사 확인

In [72]:
kiwi.analyze('고기 양도 많고 구성이 알차요')  # 양 == NNG / 알차 == VA

[([Token(form='고기', tag='NNG', start=0, len=2),
   Token(form='양', tag='NNG', start=3, len=1),
   Token(form='도', tag='JX', start=4, len=1),
   Token(form='많', tag='VA', start=6, len=1),
   Token(form='고', tag='EC', start=7, len=1),
   Token(form='구성', tag='NNG', start=9, len=2),
   Token(form='이', tag='JKS', start=11, len=1),
   Token(form='알차', tag='VA', start=13, len=2),
   Token(form='어요', tag='EF', start=14, len=2)],
  -62.784297943115234)]

In [73]:
kiwi.analyze('정말 맛있어요!')  # 맛 == VA (맛있)

[([Token(form='정말', tag='MAG', start=0, len=2),
   Token(form='맛있', tag='VA', start=3, len=2),
   Token(form='어요', tag='EF', start=5, len=2),
   Token(form='!', tag='SF', start=7, len=1)],
  -14.709687232971191)]

In [74]:
kiwi.analyze('가격이 너무 비싸요ㅠㅠ 더 싸고 가성비 좋으면 많이 시켜먹을 듯') # 가격, 가성비 == NNG / 비싸 == VA / 싸 == VV

[([Token(form='가격', tag='NNG', start=0, len=2),
   Token(form='이', tag='JKS', start=2, len=1),
   Token(form='너무', tag='MAG', start=4, len=2),
   Token(form='비싸', tag='VA', start=7, len=2),
   Token(form='어요', tag='EF', start=8, len=2),
   Token(form='ㅠㅠ', tag='SW', start=10, len=2),
   Token(form='더', tag='MAG', start=13, len=1),
   Token(form='싸', tag='VA', start=15, len=1),
   Token(form='고', tag='EC', start=16, len=1),
   Token(form='가성비', tag='NNG', start=18, len=3),
   Token(form='좋', tag='VA', start=22, len=1),
   Token(form='으면', tag='EC', start=23, len=2),
   Token(form='많이', tag='MAG', start=26, len=2),
   Token(form='시키', tag='VV', start=29, len=2),
   Token(form='어', tag='EC', start=30, len=1),
   Token(form='먹', tag='VV', start=31, len=1),
   Token(form='을', tag='ETM', start=32, len=1),
   Token(form='듯', tag='NNB', start=34, len=1)],
  -116.14079284667969)]

In [75]:
train['review']

0         사진보다 실물이 백배 맛있어요~ 고기 양도 많아요~ 배달시간도 딱 맞춰서 보내주셨어요~
1                           리뷰가 늦었네요. 맛있게 잘 먹었습니다. 또 시킬게요!
2                       잘 먹었습니다. 다만 주문에 있던 펩시콜라는 받지 못했습니다.
3                                       굿 말이 필요 없는 메뉴 였습니다
4                                              맛있어요 양도많구좋음
                               ...                        
18681                                          맛있게 잘 먹었어요 
18682            요청사항을 항상 잘 들어주셨는데 오늘은 못 모셨나봐요 그래도 잘 먹었습니다
18683                                 배달 빨라요 맛있게 잘 먹었습니당 !
18684                                              잘 먹었어요~
18685     식후감 자주 시키는데 항상 요청사항도 잘들어주시고 맛있고 빨라요 앞으로도 자주 시킬게요
Name: review, Length: 18686, dtype: object

In [76]:
kiwi_reviews = []

for review in tqdm(train['review']):
    results = []  # 결과 리스트를 각 리뷰마다 초기화
    result = kiwi.analyze(review)
    for token, pos, _, _ in result[0][0]:
        if pos in ['NNG', 'VA', 'VV', 'XR']:  # 일반명사, 형용사, 동사, 어근
            results.append(token)
    kiwi_reviews.append(results)


  0%|          | 0/18686 [00:00<?, ?it/s]

In [77]:
train['kiwi_reviews'] = kiwi_reviews

<ipython-input-77-27476ffd4693>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['kiwi_reviews'] = kiwi_reviews


In [78]:
# train['kiwi_reviews'].to_csv(f'{DATA_PATH}kiwi_reviews.csv', index=False)

In [79]:
# kiwi_reviews = pd.read_csv(f"{DATA_PATH}kiwi_reviews.csv")

In [80]:
# train = pd.concat([train,kiwi_reviews],axis=1)

In [81]:
train.head(10)

,review,reply,kiwi_reviews
0,사진보다 실물이 백배 맛있어요~ 고기 양도 많아요~ 배달시간도 딱 맞춰서 보내주셨어요~,"이렇게 저희 매장 찾아주셔서 감사드립니다! 맛 뿐만 아니라 양도, 서비스도 모두 최...","[사진, 실물, 배, 맛있, 고기, 양, 많, 배달, 시간, 맞추, 보내]"
1,리뷰가 늦었네요. 맛있게 잘 먹었습니다. 또 시킬게요!,좋은 리뷰 덕분에 기분이 행복해지는 것 같아요 고객님께서 맛있다고 말씀 주시니 피로...,"[리뷰, 늦, 맛있, 먹, 시키]"
2,잘 먹었습니다. 다만 주문에 있던 펩시콜라는 받지 못했습니다.,소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. 저희를 찾아 주신 만큼 ...,"[먹, 주문, 있, 하]"
3,굿 말이 필요 없는 메뉴 였습니다,수 많은 매장들을 헤치고 와주셔서 감사합니다! 저희 매장의 메뉴를 제대로 즐기신 거...,"[굿, 말, 필요, 없, 메뉴]"
4,맛있어요 양도많구좋음,수 많은 매장들을 헤치고 와주셔서 감사합니다 메뉴 그 자체로 행복을 드리고 싶었는데...,"[맛있, 양, 많, 좋]"
5,양많고 맛있었어요,저희 매장 찾아주신 것도 감사한데 리뷰까지.감동입니다 고객님들의 배고픔을 저희는 지...,"[양, 많, 맛있]"
6,처음 주문했는데 맛있어서 다먹었어요 리뷰 묵사발도 꼭 드세요! 맛있게 잘먹었습니다 ...,정성 가득한 리뷰 작성해주셔서 감사합니다 맛있는 시간이 되셨다니 정말 다행입니다 저...,"[주문, 맛있, 먹, 리뷰, 묵사발, 들, 맛있, 먹, 다음, 주문]"
7,가성비 아주 만족 합니다. 맛있게 잘먹었습니다.,즐거운 순간을 저희와 함께해주셔서 감사합니다 믿고 찾아 주셔서 진심으로 감사드립니다...,"[가성비, 만족, 맛있, 먹]"
8,맛있게 먹었습니다!,귀한 발걸음 감사합니다! 아낌없이 칭찬해 주셔서 기분이 좋습니다 저희 매장은 맛과 ...,"[맛있, 먹]"
9,새벽부터 시켰는데 배송 시간 맞춰서 잘 와주셨고 도시락도 맛있고 묵사발도 맛있습니다~!,귀한 발걸음 감사합니다! 고객님의 리뷰는 저희를 성장시키는 원동력이 됩니다~ 내 가...,"[새벽, 시키, 배송, 시간, 맞추, 오, 도시락, 맛있, 묵사발, 맛있]"


In [82]:
from keybert import KeyBERT

keybert_model = KeyBERT()

In [83]:

# keybert_reviews = []
# for keyword in tqdm(train['kiwi_reviews']):
#     tmp = []
#     keyword = ' '.join(keyword)
#     result = keybert_model.extract_keywords(keyword,
#                               keyphrase_ngram_range=(1, 1),
#                               use_mmr=True,
#                               diversity=0.7,
#                               stop_words=None,
#                               top_n=5)
#     tmp.append(result)
#     keybert_reviews.extend(tmp)

In [84]:
# train['keybert_reviews'] = keybert_reviews

In [85]:
# train['keybert_reviews'].to_csv(f"{DATA_PATH}keybert_reviews.csv", index=False)

In [86]:
keybert_reviews = pd.read_csv(f"{DATA_PATH}keybert_reviews.csv")

In [87]:
train = pd.concat([train,keybert_reviews],axis=1)

In [88]:
train

,review,reply,kiwi_reviews,keybert_reviews
0,사진보다 실물이 백배 맛있어요~ 고기 양도 많아요~ 배달시간도 딱 맞춰서 보내주셨어요~,"이렇게 저희 매장 찾아주셔서 감사드립니다! 맛 뿐만 아니라 양도, 서비스도 모두 최...","[사진, 실물, 배, 맛있, 고기, 양, 많, 배달, 시간, 맞추, 보내]","[('보내', 0.7185), ('실물', 0.6423), ('사진', 0.635)..."
1,리뷰가 늦었네요. 맛있게 잘 먹었습니다. 또 시킬게요!,좋은 리뷰 덕분에 기분이 행복해지는 것 같아요 고객님께서 맛있다고 말씀 주시니 피로...,"[리뷰, 늦, 맛있, 먹, 시키]","[('리뷰', 0.7255), ('시키', 0.6733), ('맛있', 0.4509)]"
2,잘 먹었습니다. 다만 주문에 있던 펩시콜라는 받지 못했습니다.,소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. 저희를 찾아 주신 만큼 ...,"[먹, 주문, 있, 하]","[('주문', 0.6778)]"
3,굿 말이 필요 없는 메뉴 였습니다,수 많은 매장들을 헤치고 와주셔서 감사합니다! 저희 매장의 메뉴를 제대로 즐기신 거...,"[굿, 말, 필요, 없, 메뉴]","[('메뉴', 0.7373), ('필요', 0.6027)]"
4,맛있어요 양도많구좋음,수 많은 매장들을 헤치고 와주셔서 감사합니다 메뉴 그 자체로 행복을 드리고 싶었는데...,"[맛있, 양, 많, 좋]","[('맛있', 0.6751)]"
...,...,...,...,...
18681,맛있게 잘 먹었어요,감사합니다! 다음에도 이용해주세용,"[맛있, 먹]","[('맛있', 0.5044)]"
18682,요청사항을 항상 잘 들어주셨는데 오늘은 못 모셨나봐요 그래도 잘 먹었습니다,죄송합니다 ! 다음에도 애용해주세용,"[요청, 사항, 오늘, 모시, 그렇, 먹]","[('요청', 0.6612), ('모시', 0.5918), ('사항', 0.5485..."
18683,배달 빨라요 맛있게 잘 먹었습니당 !,감사합니다! 다음에도 이용해주세용,"[배달, 빠르, 맛있, 먹]","[('배달', 0.7808), ('빠르', 0.4792), ('맛있', 0.4792)]"
18684,잘 먹었어요~,감사합니다! 다음에도 이용해주세용,[먹],[]


In [89]:
import re

total_keybert = []

for i in train['keybert_reviews']:
    korean_words = re.findall(r'[가-힣]+', i)  # ' ' 사이의 한글 단어 추출
    total_keybert.extend(korean_words)


In [90]:
total_keybert[:10]

['보내', '실물', '사진', '고기', '맛있', '리뷰', '시키', '맛있', '주문', '메뉴']

- Keybert 추출 결과의 Counter 탑 100 중 라벨 키워드 각각 확인

In [91]:
from collections import Counter

word_counts = Counter(total_keybert)

sorted_word_counts = sorted(word_counts.items(), key=lambda x: x[1], reverse=True)

for word, count in sorted_word_counts[:100]:
    print(f"{word}: {count}")

맛있: 11059
시키: 2134
배달: 2057
주문: 1418
감사: 1085
빠르: 1057
다음: 718
맛나: 639
서비스: 558
최고: 459
리뷰: 445
고기: 371
오늘: 359
처음: 345
깔끔: 328
만족: 304
사진: 290
번창: 262
푸짐: 257
피자: 256
생각: 241
음식: 238
사장: 232
메뉴: 225
추천: 217
그렇: 217
국물: 215
추가: 210
괜찮: 204
오랜만: 189
김치: 188
요청: 188
반찬: 175
소스: 171
가성비: 170
포장: 169
초밥: 165
치킨: 163
생각나: 158
시간: 157
친절: 155
떡볶이: 152
마라탕: 150
가격: 149
드리: 148
이벤트: 146
맛집: 142
푸짐하: 139
곱창: 136
배부르: 131
치즈: 131
커피: 129
아이: 127
김밥: 125
토핑: 121
매콤하: 121
느낌: 116
구성: 114
배송: 112
적당: 111
파스타: 111
돈까스: 109
정도: 103
계란: 100
모르: 100
족발: 100
닭발: 100
따뜻: 98
냉면: 94
단골: 90
이번: 90
느끼: 89
좋아하: 88
신선: 86
땡기: 85
사항: 79
들어가: 78
스파게티: 78
양념: 77
양도: 75
샐러드: 73
이용: 72
매장: 71
보쌈: 71
보내: 70
넉넉: 69
냄새: 69
다양: 69
새우: 69
꿔바로우: 68
튀김: 65
재료: 65
연어: 65
기분: 61
세트: 61
신선하: 61
만들: 60
다르: 60
두툼: 59
가게: 59


In [92]:
#100위 내 키워드들 중 맛, 양, 가격, 서비스, 배달 관련 키워드 추출

taste_keyword = ['맛있','맛나','맛집','매콤하','신선하','냄새']
quantity_keyword = ['양','푸짐','푸짐하','넉넉','다양','두툼','배부르']
price_keyword = ['가성비','가격']
service_keyword = ['서비스','깔끔','이벤트','친절']
delivery_keyword = ['배달','빠르','배송']

In [93]:
train['category'] = None

In [95]:
def get_category(words):
    found_categories = set()  # 중복을 제외한 카테고리를 저장할 집합

    for kiwi in words:
        if kiwi in taste_keyword:
            found_categories.add('맛')
        elif kiwi in quantity_keyword:
            found_categories.add('양')
        elif kiwi in price_keyword:
            found_categories.add('가격')
        elif kiwi in service_keyword:
            found_categories.add('서비스')
        elif kiwi in delivery_keyword:
            found_categories.add('배달')

    # 집합을 리스트로 변환
    unique_categories = list(found_categories)

    return unique_categories


In [96]:

# 'get_category' 함수를 'apply' 함수와 람다 함수를 사용하여 'category' 열에 적용
train['category'] = train['kiwi_reviews'].apply(lambda words: get_category(words))


In [97]:
train

,review,reply,kiwi_reviews,keybert_reviews,category
0,사진보다 실물이 백배 맛있어요~ 고기 양도 많아요~ 배달시간도 딱 맞춰서 보내주셨어요~,"이렇게 저희 매장 찾아주셔서 감사드립니다! 맛 뿐만 아니라 양도, 서비스도 모두 최...","[사진, 실물, 배, 맛있, 고기, 양, 많, 배달, 시간, 맞추, 보내]","[('보내', 0.7185), ('실물', 0.6423), ('사진', 0.635)...","[양, 배달, 맛]"
1,리뷰가 늦었네요. 맛있게 잘 먹었습니다. 또 시킬게요!,좋은 리뷰 덕분에 기분이 행복해지는 것 같아요 고객님께서 맛있다고 말씀 주시니 피로...,"[리뷰, 늦, 맛있, 먹, 시키]","[('리뷰', 0.7255), ('시키', 0.6733), ('맛있', 0.4509)]",[맛]
2,잘 먹었습니다. 다만 주문에 있던 펩시콜라는 받지 못했습니다.,소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. 저희를 찾아 주신 만큼 ...,"[먹, 주문, 있, 하]","[('주문', 0.6778)]",[]
3,굿 말이 필요 없는 메뉴 였습니다,수 많은 매장들을 헤치고 와주셔서 감사합니다! 저희 매장의 메뉴를 제대로 즐기신 거...,"[굿, 말, 필요, 없, 메뉴]","[('메뉴', 0.7373), ('필요', 0.6027)]",[]
4,맛있어요 양도많구좋음,수 많은 매장들을 헤치고 와주셔서 감사합니다 메뉴 그 자체로 행복을 드리고 싶었는데...,"[맛있, 양, 많, 좋]","[('맛있', 0.6751)]","[양, 맛]"
...,...,...,...,...,...
18681,맛있게 잘 먹었어요,감사합니다! 다음에도 이용해주세용,"[맛있, 먹]","[('맛있', 0.5044)]",[맛]
18682,요청사항을 항상 잘 들어주셨는데 오늘은 못 모셨나봐요 그래도 잘 먹었습니다,죄송합니다 ! 다음에도 애용해주세용,"[요청, 사항, 오늘, 모시, 그렇, 먹]","[('요청', 0.6612), ('모시', 0.5918), ('사항', 0.5485...",[]
18683,배달 빨라요 맛있게 잘 먹었습니당 !,감사합니다! 다음에도 이용해주세용,"[배달, 빠르, 맛있, 먹]","[('배달', 0.7808), ('빠르', 0.4792), ('맛있', 0.4792)]","[배달, 맛]"
18684,잘 먹었어요~,감사합니다! 다음에도 이용해주세용,[먹],[],[]


In [57]:
train

,review,reply,kiwi_reviews,keybert_reviews,category
0,사진보다 실물이 백배 맛있어요~ 고기 양도 많아요~ 배달시간도 딱 맞춰서 보내주셨어요~,"이렇게 저희 매장 찾아주셔서 감사드립니다! 맛 뿐만 아니라 양도, 서비스도 모두 최...","[사진, 실물, 배, 맛있, 고기, 양, 많, 배달, 시간, 맞추, 보내]","[('보내', 0.7185), ('실물', 0.6423), ('사진', 0.635)...","[양, 배달, 맛]"
1,리뷰가 늦었네요. 맛있게 잘 먹었습니다. 또 시킬게요!,좋은 리뷰 덕분에 기분이 행복해지는 것 같아요 고객님께서 맛있다고 말씀 주시니 피로...,"[리뷰, 늦, 맛있, 먹, 시키]","[('리뷰', 0.7255), ('시키', 0.6733), ('맛있', 0.4509)]",[맛]
2,잘 먹었습니다. 다만 주문에 있던 펩시콜라는 받지 못했습니다.,소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. 저희를 찾아 주신 만큼 ...,"[먹, 주문, 있, 하]","[('주문', 0.6778)]",[]
3,굿 말이 필요 없는 메뉴 였습니다,수 많은 매장들을 헤치고 와주셔서 감사합니다! 저희 매장의 메뉴를 제대로 즐기신 거...,"[굿, 말, 필요, 없, 메뉴]","[('메뉴', 0.7373), ('필요', 0.6027)]",[]
4,맛있어요 양도많구좋음,수 많은 매장들을 헤치고 와주셔서 감사합니다 메뉴 그 자체로 행복을 드리고 싶었는데...,"[맛있, 양, 많, 좋]","[('맛있', 0.6751)]","[양, 맛]"
...,...,...,...,...,...
18681,맛있게 잘 먹었어요,감사합니다! 다음에도 이용해주세용,"[맛있, 먹]","[('맛있', 0.5044)]",[맛]
18682,요청사항을 항상 잘 들어주셨는데 오늘은 못 모셨나봐요 그래도 잘 먹었습니다,죄송합니다 ! 다음에도 애용해주세용,"[요청, 사항, 오늘, 모시, 그렇, 먹]","[('요청', 0.6612), ('모시', 0.5918), ('사항', 0.5485...",[]
18683,배달 빨라요 맛있게 잘 먹었습니당 !,감사합니다! 다음에도 이용해주세용,"[배달, 빠르, 맛있, 먹]","[('배달', 0.7808), ('빠르', 0.4792), ('맛있', 0.4792)]","[배달, 맛]"
18684,잘 먹었어요~,감사합니다! 다음에도 이용해주세용,[먹],[],[]


In [99]:
train_ft = train[train['category'].apply(lambda x: x != '[]' and len(x) > 0)]
train_ft.shape

(14892, 5)

- TF-IDF 추출

In [39]:
# train['review'].tolist()

In [40]:
# from sklearn.feature_extraction.text import TfidfVectorizer

# # 텍스트 데이터
# corpus = train['review'].tolist()

# # TF-IDF 벡터화 객체 생성
# tfidf_vectorizer = TfidfVectorizer()

# # TF-IDF 벡터 생성
# tfidf_matrix = tfidf_vectorizer.fit_transform(corpus)

# # TF-IDF 행렬을 배열로 변환
# tfidf_array = tfidf_matrix.toarray()

# # 단어 목록 가져오기
# words = tfidf_vectorizer.get_feature_names_out()

# # TF-IDF 결과 출력
# for i, doc in enumerate(corpus):
#     print(f"Document {i + 1}:")
#     for j, word in enumerate(words):
#         print(f"{word}: {tfidf_array[i][j]:.2f}")
#     print()


In [41]:
# tfidf_df = pd.DataFrame(data=tfidf_array, columns=words)
# # Add the 'tfidf' column to the 'train' DataFrame
# train['tfidf'] = tfidf_df.values.tolist()

In [42]:
train

,review,reply,kiwi_reviews,keybert_reviews,category
0,사진보다 실물이 백배 맛있어요~ 고기 양도 많아요~ 배달시간도 딱 맞춰서 보내주셨어요~,"이렇게 저희 매장 찾아주셔서 감사드립니다! 맛 뿐만 아니라 양도, 서비스도 모두 최...","[사진, 실물, 배, 맛있, 고기, 양, 많, 배달, 시간, 맞추, 보내]","[('보내', 0.7185), ('실물', 0.6423), ('사진', 0.635)...","양, 배달, 맛"
1,리뷰가 늦었네요. 맛있게 잘 먹었습니다. 또 시킬게요!,좋은 리뷰 덕분에 기분이 행복해지는 것 같아요 고객님께서 맛있다고 말씀 주시니 피로...,"[리뷰, 늦, 맛있, 먹, 시키]","[('리뷰', 0.7255), ('시키', 0.6733), ('맛있', 0.4509)]",맛
2,잘 먹었습니다. 다만 주문에 있던 펩시콜라는 받지 못했습니다.,소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. 저희를 찾아 주신 만큼 ...,"[먹, 주문, 있, 하]","[('주문', 0.6778)]",
3,굿 말이 필요 없는 메뉴 였습니다,수 많은 매장들을 헤치고 와주셔서 감사합니다! 저희 매장의 메뉴를 제대로 즐기신 거...,"[굿, 말, 필요, 없, 메뉴]","[('메뉴', 0.7373), ('필요', 0.6027)]",
4,맛있어요 양도많구좋음,수 많은 매장들을 헤치고 와주셔서 감사합니다 메뉴 그 자체로 행복을 드리고 싶었는데...,"[맛있, 양, 많, 좋]","[('맛있', 0.6751)]","양, 맛"
...,...,...,...,...,...
18681,맛있게 잘 먹었어요,감사합니다! 다음에도 이용해주세용,"[맛있, 먹]","[('맛있', 0.5044)]",맛
18682,요청사항을 항상 잘 들어주셨는데 오늘은 못 모셨나봐요 그래도 잘 먹었습니다,죄송합니다 ! 다음에도 애용해주세용,"[요청, 사항, 오늘, 모시, 그렇, 먹]","[('요청', 0.6612), ('모시', 0.5918), ('사항', 0.5485...",
18683,배달 빨라요 맛있게 잘 먹었습니당 !,감사합니다! 다음에도 이용해주세용,"[배달, 빠르, 맛있, 먹]","[('배달', 0.7808), ('빠르', 0.4792), ('맛있', 0.4792)]","배달, 맛"
18684,잘 먹었어요~,감사합니다! 다음에도 이용해주세용,[먹],[],


- 네이버 라벨 활용 추출

In [43]:
taste_keywords = ['맛','반찬','사이드','안주','건강', '향신료', '재료', '잡내', '질']
quantity_keywords = ['양','알차']
price_keywords = ['가격', '가성비', '비싼', '가치']
service_keywords = ['친절','포장','빨리']

In [44]:
# train['naver_category'] = None

In [45]:
# for i, words in tqdm(enumerate(train['review'])):
#     found_categories = []  # 중복을 제외한 카테고리를 저장할 집합

#     for review in words:
#         if kiwi in taste_keyword:
#             found_categories.add('맛')
#         elif kiwi in quantity_keyword:
#             found_categories.add('양')
#         elif kiwi in price_keyword:
#             found_categories.add('가격')
#         elif kiwi in service_keyword:
#             found_categories.add('서비스')
#         elif kiwi in delivery_keyword:
#             found_categories.add('배달')

#     # if not found_categories:
#     #     found_categories.add('')  # Default to '서비스' if no category is found

#     # 집합을 리스트로 변환
#     # unique_categories = list(found_categories)
#     train.loc[i, 'naver_category'] = ', '.join(found_categories)  # 콤마로 구분하여 문자열로 변환하여 할당


## 키워드 라벨화

In [46]:
train_ft[['review','category']]


,review,category
0,사진보다 실물이 백배 맛있어요~ 고기 양도 많아요~ 배달시간도 딱 맞춰서 보내주셨어요~,"양, 배달, 맛"
1,리뷰가 늦었네요. 맛있게 잘 먹었습니다. 또 시킬게요!,맛
4,맛있어요 양도많구좋음,"양, 맛"
5,양많고 맛있었어요,"양, 맛"
6,처음 주문했는데 맛있어서 다먹었어요 리뷰 묵사발도 꼭 드세요! 맛있게 잘먹었습니다 ...,맛
...,...,...
18679,맛있어요 고기 양 짱 많아요,"양, 맛"
18680,"밥에 비해 양이 적어요 특히 연어는,",양
18681,맛있게 잘 먹었어요,맛
18683,배달 빨라요 맛있게 잘 먹었습니당 !,"배달, 맛"


## 모델 학습

In [100]:
from sklearn.preprocessing import MultiLabelBinarizer
# from sklearn.preprocessing import LabelBinarizer

from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier


In [48]:
# train_ft['category'] = train_ft['category'].str.split(',')
# train_ft['category']

In [102]:

# MultiLabelBinarizer 원핫 인코딩
mlb = MultiLabelBinarizer()
encoded_labels = mlb.fit_transform(train_ft['category'])
label_classes = mlb.classes_

In [103]:
encoded_labels.shape

(14892, 5)

In [104]:
encoded_labels  #train의 카테고리 부분을 모두 멀티라벨 태스크로 바꾼 것


array([[0, 1, 1, 0, 1],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 1],
       ...,
       [0, 1, 0, 0, 0],
       [0, 1, 1, 0, 0],
       [0, 1, 1, 0, 0]])

In [105]:
rf_classifier = RandomForestClassifier(n_estimators=100)
multi_output_classifier = MultiOutputClassifier(rf_classifier)

In [106]:

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, VotingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import BaggingClassifier


from sklearn.neural_network import MLPClassifier
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN, Dense, Conv1D, MaxPooling1D, Flatten
from sklearn.neighbors import KNeighborsClassifier

In [108]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, f1_score, accuracy_score
from sklearn.pipeline import Pipeline

# 데이터 불러오기
# data = pd.read_csv('reviews.csv')  # 리뷰 데이터 파일을 불러옵니다.

# 데이터 전처리
# 리뷰 텍스트를 TF-IDF 벡터로 변환합니다.
tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X = tfidf_vectorizer.fit_transform(train_ft['review'])

# 다중 라벨을 바이너리로 변환합니다.
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(train_ft['category'])

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 다중분류 모델 정의
models = {
    "Logistic Regression": LogisticRegression(),
    "Linear Support Vector Machine": LinearSVC(),
    "Linear SVM": SVC(kernel="linear"),
    "Decision Trees": DecisionTreeClassifier(),
    "Random Forests": RandomForestClassifier(),
    "Gradient Boosting: AdaBoost": AdaBoostClassifier(),
    "XGBoost": XGBClassifier(),
    "LightGBM": LGBMClassifier(),
    "Multilayer Perceptron, MLP": MLPClassifier(),
    # "Recurrent Neural Networks, RNN": Sequential([Embedding(input_dim=num_words, output_dim=32, input_length=max_len),
    #                                               SimpleRNN(32, return_sequences=True),
    #                                               SimpleRNN(32),
    #                                               Dense(3, activation='softmax')]),
    # "Convolutional Neural Networks, CNN": Sequential([Embedding(input_dim=num_words, output_dim=32, input_length=max_len),
    #                                                   Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'),
    #                                                   MaxPooling1D(pool_size=2),
    #                                                   Flatten(),
    #                                                   Dense(3, activation='softmax')]),
    "Ensemble Models": VotingClassifier(estimators=[
        ('rf', RandomForestClassifier()),
        ('gb', GradientBoostingClassifier()),
        ('xgb', XGBClassifier())
    ], voting='soft'),
    "K-Nearest Neighbors, KNN": KNeighborsClassifier()
}



In [109]:
for model_name, model in tqdm(models.items()):
    # 모델 학습
    classifier = OneVsRestClassifier(model)
    classifier.fit(X_train, y_train)

    # 모델 평가
    y_pred = classifier.predict(X_test)

    # F1 스코어와 정확도 계산
    f1_micro = f1_score(y_test, y_pred, average='micro')
    accuracy = accuracy_score(y_test, y_pred)

    # 결과 출력
    print(f"Model: {model_name}")
    print(f"F1 스코어 (Micro): {f1_micro:.2f}")
    print(f"정확도: {accuracy:.2f}")

    # 새로운 리뷰에 라벨 예측
    new_reviews = ["맛이 정말 좋아요!", "양이 부족해요.", "가격이 너무 비싸요.",'사장님이 친절해요','배달이 느려요']
    new_reviews_vectorized = tfidf_vectorizer.transform(new_reviews)
    predicted_labels = classifier.predict(new_reviews_vectorized)
    predicted_categories = mlb.inverse_transform(predicted_labels)

    for review, categories in zip(new_reviews, predicted_categories):
        print(f"리뷰: {review}")
        print(f"예측 라벨: {categories}")

    print("=" * 50)

  0%|          | 0/11 [00:00<?, ?it/s]

Model: Logistic Regression
F1 스코어 (Micro): 0.91
정확도: 0.82
리뷰: 맛이 정말 좋아요!
예측 라벨: ('맛',)
리뷰: 양이 부족해요.
예측 라벨: ('양',)
리뷰: 가격이 너무 비싸요.
예측 라벨: ('가격', '맛')
리뷰: 사장님이 친절해요
예측 라벨: ()
리뷰: 배달이 느려요
예측 라벨: ('배달',)
Model: Linear Support Vector Machine
F1 스코어 (Micro): 0.94
정확도: 0.86
리뷰: 맛이 정말 좋아요!
예측 라벨: ()
리뷰: 양이 부족해요.
예측 라벨: ('양',)
리뷰: 가격이 너무 비싸요.
예측 라벨: ('가격',)
리뷰: 사장님이 친절해요
예측 라벨: ('서비스',)
리뷰: 배달이 느려요
예측 라벨: ('배달',)
Model: Linear SVM
F1 스코어 (Micro): 0.94
정확도: 0.87
리뷰: 맛이 정말 좋아요!
예측 라벨: ()
리뷰: 양이 부족해요.
예측 라벨: ('양',)
리뷰: 가격이 너무 비싸요.
예측 라벨: ('가격',)
리뷰: 사장님이 친절해요
예측 라벨: ()
리뷰: 배달이 느려요
예측 라벨: ('배달',)
Model: Decision Trees
F1 스코어 (Micro): 0.92
정확도: 0.82
리뷰: 맛이 정말 좋아요!
예측 라벨: ()
리뷰: 양이 부족해요.
예측 라벨: ('양',)
리뷰: 가격이 너무 비싸요.
예측 라벨: ('가격',)
리뷰: 사장님이 친절해요
예측 라벨: ('배달',)
리뷰: 배달이 느려요
예측 라벨: ('배달',)
Model: Random Forests
F1 스코어 (Micro): 0.93
정확도: 0.86
리뷰: 맛이 정말 좋아요!
예측 라벨: ('맛',)
리뷰: 양이 부족해요.
예측 라벨: ('양',)
리뷰: 가격이 너무 비싸요.
예측 라벨: ('가격', '맛')
리뷰: 사장님이 친절해요
예측 라벨: ('배달',)
리뷰: 배달이 느려요
예측 라벨: ('배달',)
Model: Gradient B

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Model: Multilayer Perceptron, MLP
F1 스코어 (Micro): 0.92
정확도: 0.82
리뷰: 맛이 정말 좋아요!
예측 라벨: ('맛', '서비스')
리뷰: 양이 부족해요.
예측 라벨: ('양',)
리뷰: 가격이 너무 비싸요.
예측 라벨: ('가격',)
리뷰: 사장님이 친절해요
예측 라벨: ('배달',)
리뷰: 배달이 느려요
예측 라벨: ('배달',)
Model: Ensemble Models
F1 스코어 (Micro): 0.94
정확도: 0.86
리뷰: 맛이 정말 좋아요!
예측 라벨: ('맛',)
리뷰: 양이 부족해요.
예측 라벨: ('양',)
리뷰: 가격이 너무 비싸요.
예측 라벨: ('가격', '맛')
리뷰: 사장님이 친절해요
예측 라벨: ()
리뷰: 배달이 느려요
예측 라벨: ('배달',)
Model: K-Nearest Neighbors, KNN
F1 스코어 (Micro): 0.81
정확도: 0.68
리뷰: 맛이 정말 좋아요!
예측 라벨: ('맛',)
리뷰: 양이 부족해요.
예측 라벨: ('양',)
리뷰: 가격이 너무 비싸요.
예측 라벨: ('가격', '맛')
리뷰: 사장님이 친절해요
예측 라벨: ('배달',)
리뷰: 배달이 느려요
예측 라벨: ('맛', '배달')


## MultiLabel Model (테스트)

In [112]:
for model_name, model in tqdm(models.items()):

    classifier = OneVsRestClassifier(model)
    classifier.fit(X_train, y_train)

    # 모델 평가
    y_pred = classifier.predict(X_test)

    # F1 스코어와 정확도 계산
    f1_micro = f1_score(y_test, y_pred, average='micro')
    accuracy = accuracy_score(y_test, y_pred)

    # 결과 출력
    print(f"Model: {model_name}")
    print(f"F1 스코어 (Micro): {f1_micro:.2f}")
    print(f"정확도: {accuracy:.2f}")

    print('리뷰생성------')

    # 새로운 리뷰에 라벨 예측
    new_reviews = [
        '맛이 있어요',
        '최고에요',
        '사장님이 친절하시네요',
        '맛은 있는데 배달이 식어서 왔어요ㅠㅠ',
        '맛이 없어요',
        '이 머리카락 들어있는거 뭔가요?',
        '배달이 너무 불친절해요',
        '최악이에요',

        '이집 피자 최애 넘나 맛있어요',
        '치킨이 바삭바삭하니 맛있네요',
        '마라샹궈 넘넘 맛있고 양도 푸짐해용! 감사합니다!',
        '간만에 술한잔생각나서 연어 주문해서 먹는데 맛있네요^^ 양이 좀적어진거같은데 다음에는 추가로 시켜야 겠어요^^ 잘먹었습니당^^',
        '짜장면 배달이 너무 느리게 와서 음식이 다 불었네요..사장님 신경좀 써주세요',
        '쌀국수 먹는데 비닐 나왔네요 반품 이런건 안하지만 청결에 신경 써주세요',
        '요기 김치찜 맛있는데 가끔 좀 짜요ㅠㅠ다음엔 육수 추가로 주심 좋겠어요',

        '닭도 쫄깃하니 맛있고 떡볶이도 맛나요👍',
        '오늘은 비계가 많은거 같아요.. 양이 상대적으로 적어서 아쉬웠어요.',
        '크림이 너무 부드럽고 맛있어요, 또 시켜먹을게요!!',
        '맛있게 먹었어요 4단계로 해도 맵지는 않아요',

        '배달 빠르게 와서 맛있게 먹었습니다!',
        '완전 만족했어요 전문 블로거인데요 양쪼끔만 더 많았으면! 종종 시켜먹을게요',
        '맛은 정말 좋았지만, 편의점가려다 배달 도착한걸 발견 못해다면 다 상한 음식 먹을뻔했네요. 노크도 없고 문자도 안주시면 음식이 온지 어떻게 알으라는건지….',
        '……………..',
        '벌레 나왔어요.. 다신 안시켜먹을 것 같아요']
    new_reviews_vectorized = tfidf_vectorizer.transform(new_reviews)
    predicted_labels = classifier.predict(new_reviews_vectorized)
    predicted_categories = mlb.inverse_transform(predicted_labels)

    for review, categories in zip(new_reviews, predicted_categories):
        print(f"리뷰: {review}")
        print(f"예측 라벨: {categories}")

    print("=" * 70)


  0%|          | 0/11 [00:00<?, ?it/s]

Model: Logistic Regression
F1 스코어 (Micro): 0.91
정확도: 0.82
리뷰생성------
리뷰: 맛이 있어요
예측 라벨: ('맛',)
리뷰: 최고에요
예측 라벨: ()
리뷰: 사장님이 친절하시네요
예측 라벨: ()
리뷰: 맛은 있는데 배달이 식어서 왔어요ㅠㅠ
예측 라벨: ('배달',)
리뷰: 맛이 없어요
예측 라벨: ('맛',)
리뷰: 이 머리카락 들어있는거 뭔가요?
예측 라벨: ('맛',)
리뷰: 배달이 너무 불친절해요
예측 라벨: ('배달',)
리뷰: 최악이에요
예측 라벨: ('맛',)
리뷰: 이집 피자 최애 넘나 맛있어요
예측 라벨: ('맛',)
리뷰: 치킨이 바삭바삭하니 맛있네요
예측 라벨: ('맛',)
리뷰: 마라샹궈 넘넘 맛있고 양도 푸짐해용! 감사합니다!
예측 라벨: ('맛', '양')
리뷰: 간만에 술한잔생각나서 연어 주문해서 먹는데 맛있네요^^ 양이 좀적어진거같은데 다음에는 추가로 시켜야 겠어요^^ 잘먹었습니당^^
예측 라벨: ('맛',)
리뷰: 짜장면 배달이 너무 느리게 와서 음식이 다 불었네요..사장님 신경좀 써주세요
예측 라벨: ('배달',)
리뷰: 쌀국수 먹는데 비닐 나왔네요 반품 이런건 안하지만 청결에 신경 써주세요
예측 라벨: ('맛',)
리뷰: 요기 김치찜 맛있는데 가끔 좀 짜요ㅠㅠ다음엔 육수 추가로 주심 좋겠어요
예측 라벨: ('맛',)
리뷰: 닭도 쫄깃하니 맛있고 떡볶이도 맛나요👍
예측 라벨: ('맛',)
리뷰: 오늘은 비계가 많은거 같아요.. 양이 상대적으로 적어서 아쉬웠어요.
예측 라벨: ('맛', '양')
리뷰: 크림이 너무 부드럽고 맛있어요, 또 시켜먹을게요!!
예측 라벨: ('맛',)
리뷰: 맛있게 먹었어요 4단계로 해도 맵지는 않아요
예측 라벨: ('맛',)
리뷰: 배달 빠르게 와서 맛있게 먹었습니다!
예측 라벨: ('맛', '배달')
리뷰: 완전 만족했어요 전문 블로거인데요 양쪼끔만 더 많았으면! 종종 시켜먹을게요
예측 라벨: ('맛',)
리뷰: 맛은 정말 좋았지만, 편의점가려다 배

# B. 멀티라벨 긍부정 분석

### 입력받으면, 문장 단위로 나누기 -> 문장 단위로 검사해서 카테고리에 넣기 -> 긍부정 확인 -> 각각의 카테고리가 몇 %의 긍부정을 가지고 있는지 확인하기